# Federated Learning implementation

In [1]:
import os
import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from numpy import array
from numpy import split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM,GRU
from keras.layers import Bidirectional
from keras.layers import TimeDistributed,Dropout,RepeatVector


from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_federated as tff

import collections

C:\Users\prave\anaconda3\envs\UPPSALA_CS_20\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [2]:
#Read the station id - 8779 dataset
path = "Uppsala\\Sem 3\\Dataset\\2014-2018_combined\\8779\\KNN_8779_2015_2018.xlsx" 
file_name = os.path.join("C:\\",path)

df_2015_2018_8779 = pd.read_excel(file_name)
del df_2015_2018_8779['CO']
df_2015_2018_8779

,NO2,NOX as NO2,PM10,PM2.5,Start,Air temperature,Precipitation,Relative humidity,Air pressure,Wind direction,Wind speed
0,33.3,69.5,101.2,51.9,2015-01-01 00:00:00,3.2,0.0,100.0,1014.8,260,5.0
1,26.1,39.0,12.5,9.3,2015-01-01 01:00:00,3.6,0.0,96.0,1014.5,250,6.0
2,18.4,25.7,12.2,9.2,2015-01-01 02:00:00,3.0,0.0,93.0,1014.0,250,5.0
3,18.1,33.5,16.2,8.9,2015-01-01 03:00:00,2.8,0.0,93.0,1013.3,250,6.0
4,11.0,22.1,14.5,5.9,2015-01-01 04:00:00,2.8,0.0,93.0,1012.8,250,5.0
...,...,...,...,...,...,...,...,...,...,...,...
43819,14.5,23.5,25.4,0.8,2019-12-31 19:00:00,2.6,0.0,61.0,1020.1,270,4.0
43820,18.0,35.2,22.0,0.3,2019-12-31 20:00:00,2.4,0.0,62.0,1020.1,260,3.0
43821,18.0,32.9,26.9,2.1,2019-12-31 21:00:00,2.4,0.0,60.0,1019.9,240,3.0
43822,35.8,76.2,42.2,5.5,2019-12-31 22:00:00,2.2,0.0,62.0,1019.3,240,4.0


In [3]:
#Read the station id - 8780 dataset
path = "Uppsala\\Sem 3\\Dataset\\2014-2018_combined\\8780\\KNN_8780_2015_2018.xlsx" 
file_name = os.path.join("C:\\",path)

df_2015_2018_8780 = pd.read_excel(file_name)

del df_2015_2018_8780['CO']
del df_2015_2018_8780['Black Carbon']

df_2015_2018_8780

,NO2,NOX as NO2,PM10,PM2.5,Start,Air temperature,Precipitation,Relative humidity,Air pressure,Wind direction,Wind speed
0,36.9,101.5,32.5,15.1,2015-01-01 00:00:00,3.2,0.0,100.0,1014.8,260,5.0
1,38.6,78.3,11.0,6.4,2015-01-01 01:00:00,3.6,0.0,96.0,1014.5,250,6.0
2,26.8,49.6,13.8,6.2,2015-01-01 02:00:00,3.0,0.0,93.0,1014.0,250,5.0
3,30.5,65.8,15.2,6.5,2015-01-01 03:00:00,2.8,0.0,93.0,1013.3,250,6.0
4,20.9,37.6,15.3,6.6,2015-01-01 04:00:00,2.8,0.0,93.0,1012.8,250,5.0
...,...,...,...,...,...,...,...,...,...,...,...
43819,18.7,36.6,23.0,4.1,2019-12-31 19:00:00,2.6,0.0,61.0,1020.1,270,4.0
43820,18.5,34.9,20.1,3.9,2019-12-31 20:00:00,2.4,0.0,62.0,1020.1,260,3.0
43821,24.3,49.2,25.5,4.7,2019-12-31 21:00:00,2.4,0.0,60.0,1019.9,240,3.0
43822,32.2,69.8,27.1,4.3,2019-12-31 22:00:00,2.2,0.0,62.0,1019.3,240,4.0


In [4]:
#Read the station id - 8781 dataset
path = "Uppsala\\Sem 3\\Dataset\\2014-2018_combined\\8781\\KNN_8781_2015_2018.xlsx" 
file_name = os.path.join("C:\\",path)

df_2015_2018_8781 = pd.read_excel(file_name)

del df_2015_2018_8781['O3']
del df_2015_2018_8781['Black Carbon']
df_2015_2018_8781

,NO2,NOX as NO2,PM10,PM2.5,Start,Air temperature,Precipitation,Relative humidity,Air pressure,Wind direction,Wind speed
0,4.5,6.2,28.4,22.9,2015-01-01 00:00:00,3.2,0.0,100.0,1014.8,260,5.0
1,3.1,3.7,8.5,5.5,2015-01-01 01:00:00,3.6,0.0,96.0,1014.5,250,6.0
2,2.1,2.9,8.7,3.6,2015-01-01 02:00:00,3.0,0.0,93.0,1014.0,250,5.0
3,1.6,2.2,10.5,4.8,2015-01-01 03:00:00,2.8,0.0,93.0,1013.3,250,6.0
4,1.4,2.1,12.1,5.6,2015-01-01 04:00:00,2.8,0.0,93.0,1012.8,250,5.0
...,...,...,...,...,...,...,...,...,...,...,...
43819,4.4,4.6,6.4,2.4,2019-12-31 19:00:00,2.6,0.0,61.0,1020.1,270,4.0
43820,3.2,3.4,4.5,2.4,2019-12-31 20:00:00,2.4,0.0,62.0,1020.1,260,3.0
43821,2.7,2.6,3.0,2.4,2019-12-31 21:00:00,2.4,0.0,60.0,1019.9,240,3.0
43822,3.1,3.1,2.8,1.7,2019-12-31 22:00:00,2.2,0.0,62.0,1019.3,240,4.0


In [5]:
#Read the station id - 18644 dataset
path = "Uppsala\\Sem 3\\Dataset\\2014-2018_combined\\18644\\KNN_18644_2015_2018.xlsx" 
file_name = os.path.join("C:\\",path)

df_2015_2018_18644 = pd.read_excel(file_name)
df_2015_2018_18644

,NO2,NOX as NO2,PM10,PM2.5,Start,Air temperature,Precipitation,Relative humidity,Air pressure,Wind direction,Wind speed
0,11.0,18.8,27.9,19.8,2015-01-01 00:00:00,3.2,0.0,100.0,1014.8,260,5.0
1,15.3,25.0,15.5,4.5,2015-01-01 01:00:00,3.6,0.0,96.0,1014.5,250,6.0
2,13.4,19.8,17.1,4.6,2015-01-01 02:00:00,3.0,0.0,93.0,1014.0,250,5.0
3,9.6,13.8,15.2,5.3,2015-01-01 03:00:00,2.8,0.0,93.0,1013.3,250,6.0
4,3.5,5.8,16.3,6.6,2015-01-01 04:00:00,2.8,0.0,93.0,1012.8,250,5.0
...,...,...,...,...,...,...,...,...,...,...,...
43819,14.7,23.0,17.0,0.6,2019-12-31 19:00:00,2.6,0.0,61.0,1020.1,270,4.0
43820,9.8,15.4,9.9,-2.5,2019-12-31 20:00:00,2.4,0.0,62.0,1020.1,260,3.0
43821,9.3,14.5,8.6,-1.0,2019-12-31 21:00:00,2.4,0.0,60.0,1019.9,240,3.0
43822,12.6,18.0,12.1,0.5,2019-12-31 22:00:00,2.2,0.0,62.0,1019.3,240,4.0


In [6]:
#Copy all the clients dataset as a list 
clients = [df_2015_2018_8779,df_2015_2018_8780,df_2015_2018_8781,df_2015_2018_18644]
clients

[        NO2  NOX as NO2   PM10  PM2.5               Start  Air temperature  \
 0      33.3        69.5  101.2   51.9 2015-01-01 00:00:00              3.2   
 1      26.1        39.0   12.5    9.3 2015-01-01 01:00:00              3.6   
 2      18.4        25.7   12.2    9.2 2015-01-01 02:00:00              3.0   
 3      18.1        33.5   16.2    8.9 2015-01-01 03:00:00              2.8   
 4      11.0        22.1   14.5    5.9 2015-01-01 04:00:00              2.8   
 ...     ...         ...    ...    ...                 ...              ...   
 43819  14.5        23.5   25.4    0.8 2019-12-31 19:00:00              2.6   
 43820  18.0        35.2   22.0    0.3 2019-12-31 20:00:00              2.4   
 43821  18.0        32.9   26.9    2.1 2019-12-31 21:00:00              2.4   
 43822  35.8        76.2   42.2    5.5 2019-12-31 22:00:00              2.2   
 43823  35.8        72.2   75.6   10.1 2019-12-31 23:00:00              2.4   
 
        Precipitation  Relative humidity  Air pres

In [7]:
# Group the timestamp dataset into days format
def split_dataset(train,test):
    # split into days and restructure into windows of daily data
    train = array(split(train, len(train)/24))
    #val = array(split(val, len(val)/24))
    test = array(split(test, len(test)/24))
    return train,test

In [8]:
def create_dataset(client):
    df_2015_2019 = client.copy()
    df_2015_2019 = df_2015_2019.drop(columns=['Relative humidity','Air pressure','Wind speed','Wind direction'])
    lstm_df = df_2015_2019.copy()  #combined_data.copy()
    
    # Add time related features
    lstm_df['Weekday'] = df_2015_2019['Start'].dt.day_name()
    lstm_df['Hour'] = df_2015_2019['Start'].dt.hour

    label_encoder_1 = LabelEncoder()
    onehot_encoder_1 = OneHotEncoder(sparse=False)

    integer_encoded = label_encoder_1.fit_transform(lstm_df['Weekday'])
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    lstm_df['Weekday'] = onehot_encoder_1.fit_transform(integer_encoded)

    integer_encoded = label_encoder_1.fit_transform(lstm_df['Hour'])
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    lstm_df['Hour'] = onehot_encoder_1.fit_transform(integer_encoded)

    # Remove timestamp attribute
    date_time = lstm_df.pop('Start')

    # Use years 2015-2018 for training and first 9 months of 2019 for validation set
    train_data = lstm_df[0:41616] #lstm_df[0:24072] #train_data = lstm_df[0:6552]

    train_df = train_data[0:] #train_data[0:35064] #train_data[0:19680] #train_data[0:5088] # Training set - January - August months
    #val_df = train_data[32856:] #train_data[19680:] #train_data[5088:] # Validation set - September month

    # Use Oct-Dec of year 2019 for Testing set
    test_df = lstm_df[41616:] #lstm_df[24072:] #lstm_df[6552:]
    
    train_df,test_df = normalize_values(df_2015_2019,lstm_df,train_df,test_df)
    
    return train_df,test_df

In [33]:
# convert history into inputs and outputs format of 24 hours history and 24 hours forecast
def to_supervised(train, n_input=24, n_out=6):
    # flatten data into timestamps data format
    data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
    X, y = list(), list()
    in_start = 0
    # step over the entire history one time step at a time
    for _ in range(len(data)):
        # define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        # ensure we have enough data for this instance
        if out_end <= len(data):
            X.append(data[in_start:in_end, :])
            y.append(data[in_end:out_end, 0:4]) # Change the 2nd dimension according to the num of features to be predicted 
        # move along one time step
        in_start += 1
    return array(X), array(y)

In [34]:
def normalize_values(df_2015_2019,lstm_df,train_df,test_df):
    # Normalize the datasets using mean and std_deviation
    lst = list(df_2015_2019.columns) 
    lst.remove('Start')

    train_mean = train_df[lst].mean()
    train_std = train_df[lst].std()

    training_set = (train_df[lst] - train_mean) / train_std
    testing_set = (test_df[lst] - train_mean) / train_std

    time_ftrs = set(list(lstm_df.columns)) - set(lst)
    time_ftrs = list(time_ftrs)

    training_set[time_ftrs] = train_df[time_ftrs]
    testing_set[time_ftrs] = test_df[time_ftrs]

    columnsTitles = list(sorted(set(lst), key=lst.index) + sorted(set(time_ftrs), key=time_ftrs.index))

    training_set = training_set.reindex(columns=columnsTitles)
    testing_set = testing_set.reindex(columns=columnsTitles)
    
    training_df,testing_df = split_dataset(training_set.values,testing_set.values)
    
    return training_df,testing_df

In [35]:
NUM_EPOCHS = 50
BATCH_SIZE = 80
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

def preprocess(dataset):

  def batch_format_fn(element):
      return collections.OrderedDict(x=element['x'], y=element['y'])

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

In [36]:
# produce datasets for each origin
def make_federated_data(clients):
    train_datasets = []
    test_datasets = []

    for client in clients:
        
        train_df,val_df,test_df = create_dataset(client)
        
        train_x, train_y = to_supervised(train_df, n_input=24)
        test_x,test_y = to_supervised(test_df,n_input=24)
        
        n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
        
        train_dataset = tf.data.Dataset.from_tensor_slices(
            ({'x': train_x, 'y': train_y}))

        test_dataset = tf.data.Dataset.from_tensor_slices(
            ({'x': test_x, 'y': test_y}))
        
        preprocessed_train_dataset = preprocess(train_dataset)
        preprocessed_test_dataset = preprocess(test_dataset)

        train_datasets.append(preprocessed_train_dataset)
        test_datasets.append(preprocessed_test_dataset)
        
    return train_datasets, test_datasets,n_timesteps, n_features, n_outputs

In [37]:
# Build the LSTM model architecture and train the model
def build_model(n_timesteps, n_features, n_outputs):
        
    # define the model parameters
    n_out_features = 4
    # define the model architecture
    model = Sequential()
    
    # Multi features predictions
    model.add(LSTM(50, activation='relu',
              input_shape=(n_timesteps, n_features)))
    model.add(Dropout(0.50))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.50))
    model.add(Dense(n_outputs*n_out_features))
    model.add(tf.keras.layers.Reshape([n_outputs, n_out_features]))  # Reshape the output layer back into [Timesteps,out_featrs]
    
    return model

In [38]:
train_datasets, test_datasets,n_timesteps, n_features, n_outputs = make_federated_data(clients)
test_datasets

[<PrefetchDataset shapes: OrderedDict([(x, (None, 24, 8)), (y, (None, 6, 4))]), types: OrderedDict([(x, tf.float64), (y, tf.float64)])>,
 <PrefetchDataset shapes: OrderedDict([(x, (None, 24, 8)), (y, (None, 6, 4))]), types: OrderedDict([(x, tf.float64), (y, tf.float64)])>,
 <PrefetchDataset shapes: OrderedDict([(x, (None, 24, 8)), (y, (None, 6, 4))]), types: OrderedDict([(x, tf.float64), (y, tf.float64)])>,
 <PrefetchDataset shapes: OrderedDict([(x, (None, 24, 8)), (y, (None, 6, 4))]), types: OrderedDict([(x, tf.float64), (y, tf.float64)])>]

In [39]:
def create_tff_model():
  return tff.learning.from_keras_model(build_model(n_timesteps, n_features, n_outputs), 
                                       input_spec=train_datasets[0].element_spec,
                                       loss= tf.keras.losses.MeanAbsoluteError(),
                                       metrics= [tf.keras.metrics.MeanAbsoluteError()])

In [40]:
print("Create averaging process")
iterative_process = tff.learning.build_federated_averaging_process(model_fn=create_tff_model,
                                                                   client_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=0.002))

Create averaging process


In [17]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

You should consider upgrading via the 'c:\users\prave\anaconda3\envs\uppsala_cs_20\python.exe -m pip install --upgrade pip' command.


In [ ]:
print("Initzialize averaging process")
state = iterative_process.initialize()

print("Start iterations")
for round_num in range(10):
  state, metrics = iterative_process.next(state, train_datasets)
  print('round{:2d},metrics={}'.format(round_num,metrics))

Initzialize averaging process
Start iterations
round 0,metrics=OrderedDict([('broadcast', ()), ('aggregation', ()), ('train', OrderedDict([('mean_absolute_error', 0.63933706), ('loss', 0.6393355)]))])
round 1,metrics=OrderedDict([('broadcast', ()), ('aggregation', ()), ('train', OrderedDict([('mean_absolute_error', 0.5848501), ('loss', 0.58485013)]))])
round 2,metrics=OrderedDict([('broadcast', ()), ('aggregation', ()), ('train', OrderedDict([('mean_absolute_error', 0.5619296), ('loss', 0.5619288)]))])
round 3,metrics=OrderedDict([('broadcast', ()), ('aggregation', ()), ('train', OrderedDict([('mean_absolute_error', 0.54879403), ('loss', 0.54879504)]))])


In [31]:
# Global model evaluated over all clients
evaluation = tff.learning.build_federated_evaluation(model_fn=create_tff_model)
test_metrics = evaluation(state.model, test_datasets)
print(test_metrics)

OrderedDict([('mean_absolute_error', 0.5345366), ('loss', 0.5345366)])


In [32]:
# Global model evaluated per individual client
for i in range(len(test_datasets)): 
    test_metrics = evaluation(state.model, [test_datasets[i]])
    print(test_metrics)

OrderedDict([('mean_absolute_error', 0.52270657), ('loss', 0.5227068)])
OrderedDict([('mean_absolute_error', 0.5586187), ('loss', 0.5586189)])
OrderedDict([('mean_absolute_error', 0.53248096), ('loss', 0.5324812)])
OrderedDict([('mean_absolute_error', 0.5243395), ('loss', 0.52434003)])
